In [13]:
# 如果尚未安装必要的库，请取消以下行的注释并运行
# !pip install requests opuslib simpleaudio numpy librosa

import requests
import opuslib
import simpleaudio as sa
import queue
import threading

# 定义服务器端点和请求负载
url = 'http://0.0.0.0:8000/text2speech'  # 根据实际情况修改
payload = {
    "text": "深夜獨行於荒蕪的小巷，忽聞身後傳來詭異的腳步聲，我寒毛直竪，心跳如雷，無法抑制對未知危險的深深恐懼。",
    "instruction": "A female speaker with high pitch, normal speaking rate, and happy emotion.",
    "seed": 12345,
    "spk_id": "中文女",  # 确保这是服务器端支持的发音人ID
    "speed": 1.0
}

# 初始化 Opus 解码器
decoder = opuslib.Decoder(16000, 1)  # 采样率16000 Hz，单声道

# 创建一个队列用于在线程之间传递PCM数据
pcm_queue = queue.Queue()

def fetch_and_decode(url, payload, pcm_queue):
    """
    发送POST请求到服务器，接收流式Opus数据，解码并放入队列。
    """
    try:
        with requests.post(url, json=payload, stream=True) as response:
            if response.status_code != 200:
                print(f"请求失败：{response.status_code} - {response.text}")
                return
            
            buffer = b''  # 初始化缓冲区
            for chunk in response.iter_content(chunk_size=1024):
                if not chunk:
                    break
                buffer += chunk
                while True:
                    if len(buffer) < 2:
                        # 不足两个字节，继续读取
                        break
                    # 读取前两个字节获取帧长度（小端格式）
                    frame_length = int.from_bytes(buffer[:2], byteorder='little')
                    if len(buffer) < 2 + frame_length:
                        # 数据不足以读取整个帧，继续读取
                        break
                    # 提取Opus帧数据
                    opus_data = buffer[2:2+frame_length]
                    buffer = buffer[2+frame_length:]  # 移除已读取的数据
                    try:
                        # 解码Opus数据为PCM
                        pcm_data = decoder.decode(opus_data, 320)  # 320帧对应20ms
                        # 转换为numpy数组并归一化到[-1.0, 1.0]
                        #pcm_float = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32) / 32767.0
                        # 将PCM数据放入队列
                        pcm_queue.put(pcm_data)
                    except Exception as e:
                        print(f"解码错误：{e}")
    except Exception as e:
        print(f"请求或解码过程中发生错误：{e}")
    finally:
        # 在队列中放入None表示播放结束
        pcm_queue.put(None)
    play_audio(pcm_queue)
    
def play_audio(pcm_queue):
    """
    从队列中获取PCM数据并播放。
    """
    try:
        buffer = b''  # 初始化缓冲区
        while True:
            data = pcm_queue.get()
            if data is None:
                break  # 播放结束
            buffer += data
            # 将PCM数据转换为16位整数
            #pcm_int16 = (data * 32767).astype(np.int16)
            # 播放PCM数据
        play_obj = sa.play_buffer(buffer, 1, 2, 16000)
        play_obj.wait_done()
    except Exception as e:
        print(f"播放过程中发生错误：{e}")

# 启动解码线程
decode_thread = threading.Thread(target=fetch_and_decode, args=(url, payload, pcm_queue))
decode_thread.start()


# 等待线程结束
decode_thread.join()
#play_thread.join()

print("音频播放完成。")

音频播放完成。
